In [ ]:
# Importing libraries

# !pip install numpy
# !pip install tensorflow
import os
# print(os.getcwd())
# print(os.listdir())
import csv
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import random
import json



# Step 1: Reading and preprocessing the data

with open('intents.json') as file:
    data = json.load(file)

lemmatizer = WordNetLemmatizer()
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        tokenized_words = nltk.word_tokenize(pattern)
       # print(tokenized_words)
        words.extend(tokenized_words)
        docs_x.append(tokenized_words)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w != '?']
words = sorted(list(set(words)))
labels = sorted(labels)

# Step 2: Creating training and testing data
training_data = []
output_data = []
out_empty = [0] * len(labels)

for x, doc in enumerate(docs_x):
    bag = []

    for w in words:
        if w in doc:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training_data.append(bag)
    output_data.append(output_row)

training_data = np.array(training_data)
output_data = np.array(output_data)

# Step 3: Creating the model
model = Sequential()
model.add(Dense(128, input_shape=(len(training_data[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(output_data[0]), activation='softmax'))

# Step 4: Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 5: Training the model
model.fit(training_data, output_data, epochs=200, batch_size=5, verbose=1)

# Step 6: Saving the model
model.save('chatbot_model.h5')

# Step 7: Loading the model
model = tf.keras.models.load_model('chatbot_model.h5')



def calculate_attendance_percentage():
    file_path = 'attendance.csv'
    student_id = input('Enter the rollno :-')
    total_classes = 0
    attended_classes = 0

    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)  # Skip the header row

        # Assuming the CSV file has columns: StudentID, Date, Attendance
        for row in csv_reader:
            if row[0] == student_id:
                total_classes += 1
                if row[2].lower() == 'present':
                    attended_classes += 1

    if total_classes == 0:
        return 0.0

    attendance_percentage = (attended_classes / total_classes) * 100
    
    print(attendance_percentage)


def print_timetable():
    
    day = input("Enter the day: ")
    department = input("Enter the department: ")
    with open('timetable.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if row[0].lower() == day.lower() and row[3].lower() == department.lower():
                print(f"Day: {row[0]}\nTime Slot: {row[1]}\nSubject: {row[2]}\nDepartment: {row[3]}\n")
                






# Step 8: Defining the chatbot function
def chatbot():
    print("Welcome to the chatbot! Start talking with the bot (type 'quit' to stop):")
    while True:
        user_input  = input("You :")
        if user_input.lower() == 'quit':
            break
        
        tokenized_words = nltk.word_tokenize(user_input)
        tokenized_words = [lemmatizer.lemmatize(w.lower()) for w in tokenized_words]
        
        input_data = np.array([0] * len(words))
        target_words = ['attendance', 'timetable']
        for word in tokenized_words:
            if word == target_words[0]:
                calculate_attendance_percentage()
            elif word == target_words[1]:
                print_timetable()
            else:
                break
        for w in tokenized_words:
            if w in words:
                input_data[words.index(w)] = 1

        results = model.predict(np.array([input_data]))
        results_index = np.argmax(results)
        tag = labels[results_index]

        for intent in data['intents']:
            if intent['tag'] == tag:
                responses = intent['responses']

        print(random.choice(responses))

# Running the chatbot function
chatbot()


In [ ]:
import csv
import random

def generate_dummy_timetable():
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    time_slots = ['9:00 AM - 10:30 AM', '10:45 AM - 12:15 PM', '1:00 PM - 2:30 PM', '2:45 PM - 4:15 PM', '4:30 PM - 6:00 PM']
    subjects = ['Mathematics', 'Physics', 'Chemistry', 'Computer Science', 'English', 'History']
    departments = ['Department A', 'Department B', 'Department C', 'Department D']

    timetable = []

    for day in days_of_week:
        for time_slot in time_slots:
            subject = random.choice(subjects)
            department = random.choice(departments)

            timetable.append([day, time_slot, subject, department])

    return timetable

def save_timetable_to_csv(file_path, timetable):
    with open(file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Day', 'Time Slot', 'Subject', 'Department'])
        csv_writer.writerows(timetable)

# Example usage:
file_path = 'timetable.csv'
dummy_timetable = generate_dummy_timetable()
save_timetable_to_csv(file_path, dummy_timetable)
print(f"Dummy timetable saved to '{file_path}'.")


In [ ]:
import csv
import random

def generate_dummy_dataset(num_students, num_records):
    attendance_records = []

    for _ in range(num_records):
        student_id = f'21bce{random.randint(1, num_students)}'
        date = f'{random.randint(1, 12)}/{random.randint(1, 28)}/2023'
        attendance_status = random.choice(['Present', 'Absent'])
        attendance_records.append([student_id, date, attendance_status])

    return attendance_records

def save_dataset_to_csv(file_path, dataset):
    with open(file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['StudentID', 'Date', 'Attendance'])
        csv_writer.writerows(dataset)

# Example usage:
num_students = 50
num_records = 100
file_path = 'attendance.csv'

attendance_dataset = generate_dummy_dataset(num_students, num_records)
save_dataset_to_csv(file_path, attendance_dataset)
print(f"Dummy dataset with {num_records} attendance records saved to '{file_path}'.")


In [ ]:
import csv

def print_timetable(day, department):
    user_day = input("Enter the day: ")
    user_department = input("Enter the department: ")
    with open('timetable.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if row[0].lower() == day.lower() and row[3].lower() == department.lower():
                print(f"Day: {row[0]}\nTime Slot: {row[1]}\nSubject: {row[2]}\nDepartment: {row[3]}\n")

# Ask for user input


# Print the timetable
print_timetable(user_day, user_department)



In [ ]:
import nltk
from nltk.corpus import stopwords

def remove_unimportant_words(sentence):
    # nltk.download('stopwords')  # Download stopwords dataset if not already downloaded
    stop_words = set(stopwords.words('english'))

    tokenized_words = nltk.word_tokenize(sentence)
    important_words = []

    for word in tokenized_words:
        if word.lower() not in stop_words:
            important_words.append(word)

    cleaned_sentence = ' '.join(important_words)

    if 'attendance' in important_words:
        print("The word 'attendance' is present in the important words.")

    return cleaned_sentence
sentence=input()
remove_unimportant_words(sentence)

In [ ]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup

def read_blog_from_website(query):
    search_results = search(query, num_results=5)  # Perform a Google search and get the top 5 results

    blog_content = []  # List to store the extracted blog content

    for result in search_results:
        try:
            response = requests.get(result)  # Fetch the webpage
            soup = BeautifulSoup(response.text, 'html.parser')  # Create a BeautifulSoup object to parse the HTML
            # Extract the blog content based on the specific HTML structure of the website
            # Adjust the CSS selectors according to the structure of the target website
            content = soup.select('div.blog-content p')
            if content:
                blog_content.append(' '.join([p.get_text() for p in content]))  # Append the extracted content to the list
        except requests.exceptions.RequestException:
            continue

    return blog_content

# Example usage
query = input("Enter a query: ")
blog_content = read_blog_from_website(query)
print(blog_content)
